In [53]:
import pandas as pd
import os
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.applications.xception import Xception, preprocess_input
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, roc_curve
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import class_weight
import keras
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Input,Dense, Dropout, Activation, Flatten, Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers.core import Dropout
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

## Engineering Features

In [ ]:
feat = os.path.join("bird_data","features")
mfcc_train_folder = os.path.join(feat,"mfcc_features", "train")
mfcc_test_folder = os.path.join(feat,"mfcc_features", "test")

# Convolutional Neural Network

In [1]:
# Batch and target size
BATCH_SIZE = 16
TARGET_SIZE = (720,288)


In [8]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(720, 288, 3)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 286, 286, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 284, 284, 64)      18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 142, 142, 64)      0         
_________________________________________________________________
dropout (Dropout)            (None, 142, 142, 64)      0         
_________________________________________________________________
flatten (Flatten)            (None, 1290496)           0         
_________________________________________________________________
dense (Dense)                (None, 64)                82591808  
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0

In [9]:
early_stopping = EarlyStopping(monitor='val_loss', patience=50, verbose=1)
callbacks = [early_stopping]

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,  
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.1,
                                   fill_mode='nearest',
                                   horizontal_flip = False)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
test_set = test_datagen.flow_from_directory(mfcc_test_folder, target_size = TARGET_SIZE, batch_size = BATCH_SIZE, class_mode = 'categorical')

In [ ]:
val_set = test_datagen.flow_from_directory(mfcc_train_folder, target_size = TARGET_SIZE, batch_size = BATCH_SIZE, class_mode = 'categorical')

In [ ]:
class_weights = class_weight.compute_class_weight('balanced', np.unique(training_set.classes), training_set.classes)

In [ ]:
model.fit_generator(training_set, epochs=10, class_weight=class_weights,  validation_data = test_set, verbose=True, callbacks=callbacks, shuffle=True)

In [ ]:
model.save('Bird_Classifier.h5')

In [ ]:
Y_pred = model.predict_generator(val_set)
y_pred = np.argmax(Y_pred, axis=1)
print(classification_report(val_set.classes, y_pred))

In [ ]:
score = model.evaluate_generator(training_set, verbose=0)
print("Training Accuracy: ", score[1])

In [ ]:
score = model.evaluate_generator(test_set, verbose=0)
print("Testing Accuracy: ", score[1])

In [ ]:
score = model.evaluate_generator(val_set, verbose=0)
print("Validating Accuracy: ", score[1])